## This document is meant to compare the output of graphium-2 vs graphium-3 datamodules

In [1]:
import graphium
import hydra
from hydra.core.global_hydra import GlobalHydra
from omegaconf import OmegaConf
import os
from os.path import dirname, realpath
from graphium.config._loader import load_accelerator, load_datamodule
import shutil

c:\Users\Dominique.Beaini\AppData\Local\miniconda3\envs\graphium\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
os.chdir(dirname(dirname(realpath(graphium.__file__))))

In [3]:

def initialize_hydra(config_path, job_name="app"):
    if GlobalHydra.instance().is_initialized():
        GlobalHydra.instance().clear()
    hydra.initialize(config_path=config_path, job_name=job_name)

def compose_main_config(config_dir):
    initialize_hydra(config_dir)
    # Compose the main configuration
    main_config = hydra.compose(config_name="main")
    return main_config

In [4]:

# Load the main configuration for toymix
CONFIG_DIR = "../../expts/hydra-configs/"
cfg = compose_main_config(CONFIG_DIR)
cfg = OmegaConf.to_container(cfg, resolve=True)
cfg.pop("tasks")

# Adapt the configuration to reduce the time it takes to run the test, less samples, less epochs
cfg["constants"]["max_epochs"] = 4
TINY_DIR = "expts/data/neurips2023/tiny-dataset/"
cfg["constants"]["data_dir"] = TINY_DIR
cfg["trainer"]["trainer"]["check_val_every_n_epoch"] = 1
cfg["trainer"]["trainer"]["max_epochs"] = 4

cfg["datamodule"]["args"]["processed_graph_data_path"] = "datacache/testing_feats"
cfg["datamodule"]["args"]["batch_size_training"] = 1
cfg["datamodule"]["args"]["batch_size_inference"] = 1
cfg["datamodule"]["args"]["task_specific_args"]["qm9"]["df_path"] = TINY_DIR + "qm9.csv.gz"
cfg["datamodule"]["args"]["task_specific_args"]["qm9"]["splits_path"] = TINY_DIR + "qm9_random_splits.pt"
cfg["datamodule"]["args"]["task_specific_args"]["tox21"]["df_path"] = TINY_DIR + "Tox21-7k-12-labels.csv.gz"
cfg["datamodule"]["args"]["task_specific_args"]["tox21"]["splits_path"] = TINY_DIR + "Tox21_random_splits.pt"
cfg["datamodule"]["args"]["task_specific_args"]["zinc"]["df_path"] = TINY_DIR + "ZINC12k.csv.gz"
cfg["datamodule"]["args"]["task_specific_args"]["zinc"]["splits_path"] = TINY_DIR + "ZINC12k_random_splits.pt"

# Initialize the accelerator
cfg, accelerator_type = load_accelerator(cfg)

# Load and initialize the dataset
datamodule = load_datamodule(cfg, accelerator_type)


C:\Users\Dominique.Beaini\AppData\Local\Temp\ipykernel_47064\1475485328.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(config_path=config_path, job_name=job_name)


In [9]:
datamodule.prepare_data()
datamodule.setup("fit")
datamodule

2025-02-04 13:15:12.559 | INFO     | graphium.data.datamodule:prepare_data:987 - Data is already prepared.
2025-02-04 13:15:12.561 | INFO     | graphium.data.datamodule:setup:1190 - -------------------
MultitaskDataset
	about = training set
	num_graphs_total = 15
	num_nodes_total = 267
	max_num_nodes_per_graph = 38
	min_num_nodes_per_graph = 8
	std_num_nodes_per_graph = 9.382252750095079
	mean_num_nodes_per_graph = 17.8
	num_edges_total = 570
	max_num_edges_per_graph = 78
	min_num_edges_per_graph = 16
	std_num_edges_per_graph = 20.199009876724155
	mean_num_edges_per_graph = 38.0
-------------------

2025-02-04 13:15:12.563 | INFO     | graphium.data.datamodule:setup:1191 - -------------------
MultitaskDataset
	about = validation set
	num_graphs_total = 30
	num_nodes_total = 469
	max_num_nodes_per_graph = 38
	min_num_nodes_per_graph = 7
	std_num_nodes_per_graph = 8.244122485477492
	mean_num_nodes_per_graph = 15.633333333333333
	num_edges_total = 996
	max_num_edges_per_graph = 84
	min_nu

AttributeError: 'MultitaskFromSmilesDataModule' object has no attribute 'num_node_feats_with_positional_encoding'

In [6]:
val_loader = datamodule.val_dataloader()
val_loader

In [7]:
iter_loader = iter(val_loader)
elems = [next(iter_loader) for _ in range(30)]

for elem in elems:
    print(elem)

{'labels': DataBatch(graph_zinc=[1, 3], x=[24, 1], edge_index=[2, 52], graph_qm9=[1, 19], graph_tox21=[1, 12], batch=[24], ptr=[2]), 'features': DataBatch(edge_index=[2, 52], edge_weight=[52], num_nodes=24, feat=[24, 85], edge_feat=[52, 13], laplacian_eigvec=[24, 8], laplacian_eigval=[24, 8], rw_return_probs=[24, 16], batch=[24], ptr=[2])}
{'labels': DataBatch(graph_zinc=[1, 3], x=[21, 1], edge_index=[2, 46], graph_tox21=[1, 12], graph_qm9=[1, 19], batch=[21], ptr=[2]), 'features': DataBatch(edge_index=[2, 46], edge_weight=[46], num_nodes=21, feat=[21, 85], edge_feat=[46, 13], laplacian_eigvec=[21, 8], laplacian_eigval=[21, 8], rw_return_probs=[21, 16], batch=[21], ptr=[2])}
{'labels': DataBatch(graph_tox21=[1, 12], x=[23, 1], edge_index=[2, 50], graph_zinc=[1, 3], graph_qm9=[1, 19], batch=[23], ptr=[2]), 'features': DataBatch(edge_index=[2, 50], edge_weight=[50], num_nodes=23, feat=[23, 85], edge_feat=[50, 13], laplacian_eigvec=[23, 8], laplacian_eigval=[23, 8], rw_return_probs=[23, 1

In [13]:
# Pickle the list of graphs
import pickle
with open("../graphium/expts/notebooks/val_graphs_graphium2_new.pkl", "wb") as f:
    pickle.dump(elems, f)

# Load the list of graphs to make sure the pickle works
with open("../graphium/expts/notebooks/val_graphs_graphium2_new.pkl", "rb") as f:
    elems = pickle.load(f)
for elem in elems:
    print(elem)

{'labels': DataBatch(graph_zinc=[1, 3], x=[24, 1], edge_index=[2, 52], graph_qm9=[1, 19], graph_tox21=[1, 12], batch=[24], ptr=[2]), 'features': DataBatch(edge_index=[2, 52], edge_weight=[52], num_nodes=24, feat=[24, 85], edge_feat=[52, 13], laplacian_eigvec=[24, 8], laplacian_eigval=[24, 8], rw_return_probs=[24, 16], batch=[24], ptr=[2])}
{'labels': DataBatch(graph_zinc=[1, 3], x=[21, 1], edge_index=[2, 46], graph_tox21=[1, 12], graph_qm9=[1, 19], batch=[21], ptr=[2]), 'features': DataBatch(edge_index=[2, 46], edge_weight=[46], num_nodes=21, feat=[21, 85], edge_feat=[46, 13], laplacian_eigvec=[21, 8], laplacian_eigval=[21, 8], rw_return_probs=[21, 16], batch=[21], ptr=[2])}
{'labels': DataBatch(graph_tox21=[1, 12], x=[23, 1], edge_index=[2, 50], graph_zinc=[1, 3], graph_qm9=[1, 19], batch=[23], ptr=[2]), 'features': DataBatch(edge_index=[2, 50], edge_weight=[50], num_nodes=23, feat=[23, 85], edge_feat=[50, 13], laplacian_eigvec=[23, 8], laplacian_eigval=[23, 8], rw_return_probs=[23, 1